Are college students dropping out?
What universities are doing "well"?

In [20]:
import polars as pl
import os
from pathlib import Path

In [21]:
data_dir_path = Path('data')

merged_pp_2021 = data_dir_path / 'data' / 'MERGED2021_22_PP.csv'
import pandas as pd

df = pd.read_csv(merged_pp_2021)
df

/var/folders/fk/07csw28s7qj2xcz4_f9th0hh0000gn/T/ipykernel_22886/2958067021.py:6: DtypeWarning: Columns (9,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984,2403,2404,2495,2496,2497,2498,2499,2500,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2527,2528,2529,2530,3215,3231) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(merged_pp_2021)


,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,OMAWDP8_NOPELL_FIRSTTIME,OMENRUP_NOPELL_FIRSTTIME,OMENRYP_NOPELL_NOTFIRSTTIME,OMENRAP_NOPELL_NOTFIRSTTIME,OMAWDP8_NOPELL_NOTFIRSTTIME,OMENRUP_NOPELL_NOTFIRSTTIME,OMACHT8_NOPELL_ALL,OMACHT8_NOPELL_FIRSTTIME,OMACHT8_NOPELL_NOTFIRSTTIME,ADDR
0,100654,100200.0,1002.0,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,...,0.3187,0.2709,0.0128,0.2949,0.4744,0.2179,329.0,251.0,78.0,4900 Meridian Street
1,100663,105200.0,1052.0,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,...,0.6937,0.0660,0.0111,0.2636,0.5136,0.2117,2358.0,1182.0,1176.0,Administration Bldg Suite 1070
2,100690,2503400.0,25034.0,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,https://www.amridgeuniversity.edu/,https://www2.amridgeuniversity.edu:9091/,...,0.0000,0.5000,0.0000,0.3333,0.4583,0.2083,26.0,2.0,24.0,1200 Taylor Rd
3,100706,105500.0,1055.0,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu/,finaid.uah.edu/,...,0.6471,0.0941,0.0082,0.2647,0.5948,0.1324,1122.0,510.0,612.0,301 Sparkman Dr
4,100724,100500.0,1005.0,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,...,0.4381,0.2167,0.0000,0.1444,0.3667,0.4889,510.0,420.0,90.0,915 S Jackson Street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6538,49576720,332918.0,3329.0,Pennsylvania State University-Penn State York,York,PA,17403-3326,Middle States Commission on Higher Education,york.psu.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6539,49576721,332919.0,3329.0,Pennsylvania State University-Penn State Great...,Malvern,PA,19355-1488,Middle States Commission on Higher Education,greatvalley.psu.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6540,49576722,332920.0,3329.0,Pennsylvania State University-Penn State Harri...,Middletown,PA,17057-4846,Middle States Commission on Higher Education,harrisburg.psu.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6541,49576723,332921.0,3329.0,Pennsylvania State University-Penn State Brand...,Media,PA,19063-5522,Middle States Commission on Higher Education,brandywine.psu.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
small_df = df[['INSTNM', 'CITY', 'STABBR', 'ACCREDAGENCY', 'SAT_AVG_ALL']]
small_df


,INSTNM,CITY,STABBR,ACCREDAGENCY,SAT_AVG_ALL
0,Alabama A & M University,Normal,AL,Southern Association of Colleges and Schools C...,954.0
1,University of Alabama at Birmingham,Birmingham,AL,Southern Association of Colleges and Schools C...,1266.0
2,Amridge University,Montgomery,AL,Southern Association of Colleges and Schools C...,NaN
3,University of Alabama in Huntsville,Huntsville,AL,Southern Association of Colleges and Schools C...,1300.0
4,Alabama State University,Montgomery,AL,Southern Association of Colleges and Schools C...,955.0
...,...,...,...,...,...
6538,Pennsylvania State University-Penn State York,York,PA,Middle States Commission on Higher Education,1260.0
6539,Pennsylvania State University-Penn State Great...,Malvern,PA,Middle States Commission on Higher Education,1260.0
6540,Pennsylvania State University-Penn State Harri...,Middletown,PA,Middle States Commission on Higher Education,1260.0
6541,Pennsylvania State University-Penn State Brand...,Media,PA,Middle States Commission on Higher Education,1260.0


In [23]:
small_df = small_df.dropna(subset=['SAT_AVG_ALL'])
small_df

,INSTNM,CITY,STABBR,ACCREDAGENCY,SAT_AVG_ALL
0,Alabama A & M University,Normal,AL,Southern Association of Colleges and Schools C...,954.0
1,University of Alabama at Birmingham,Birmingham,AL,Southern Association of Colleges and Schools C...,1266.0
3,University of Alabama in Huntsville,Huntsville,AL,Southern Association of Colleges and Schools C...,1300.0
4,Alabama State University,Montgomery,AL,Southern Association of Colleges and Schools C...,955.0
5,The University of Alabama,Tuscaloosa,AL,Southern Association of Colleges and Schools C...,1244.0
...,...,...,...,...,...
6537,Pennsylvania State University-Penn State Wilke...,Lehman,PA,Middle States Commission on Higher Education,1260.0
6538,Pennsylvania State University-Penn State York,York,PA,Middle States Commission on Higher Education,1260.0
6539,Pennsylvania State University-Penn State Great...,Malvern,PA,Middle States Commission on Higher Education,1260.0
6540,Pennsylvania State University-Penn State Harri...,Middletown,PA,Middle States Commission on Higher Education,1260.0


In [14]:
import duckdb

db = duckdb.read_csv(merged_pp_2021)
db

┌──────────┬──────────┬─────────┬───┬──────────────────────┬──────────────────────┬──────────────────────┐
│  UNITID  │  OPEID   │ OPEID6  │ … │ OMACHT8_NOPELL_FIR…  │ OMACHT8_NOPELL_NOT…  │         ADDR         │
│  int64   │ varchar  │ varchar │   │       varchar        │       varchar        │       varchar        │
├──────────┼──────────┼─────────┼───┼──────────────────────┼──────────────────────┼──────────────────────┤
│   100654 │ 00100200 │ 001002  │ … │ 251                  │ 78                   │ 4900 Meridian Street │
│   100663 │ 00105200 │ 001052  │ … │ 1182                 │ 1176                 │ Administration Bld…  │
│   100690 │ 02503400 │ 025034  │ … │ 2                    │ 24                   │ 1200 Taylor Rd       │
│   100706 │ 00105500 │ 001055  │ … │ 510                  │ 612                  │ 301 Sparkman Dr      │
│   100724 │ 00100500 │ 001005  │ … │ 420                  │ 90                   │ 915 S Jackson Street │
│   100751 │ 00105100 │ 001051  │ … │

In [15]:
cols = db.execute("SELECT * FROM information_schema.columns").fetchall()
cols

TypeError: execute(): incompatible function arguments. The following argument types are supported:
    1. (self: duckdb.duckdb.DuckDBPyRelation) -> duckdb.duckdb.DuckDBPyRelation

Invoked with: ┌──────────┬──────────┬─────────┬───┬──────────────────────┬──────────────────────┬──────────────────────┐
│  UNITID  │  OPEID   │ OPEID6  │ … │ OMACHT8_NOPELL_FIR…  │ OMACHT8_NOPELL_NOT…  │         ADDR         │
│  int64   │ varchar  │ varchar │   │       varchar        │       varchar        │       varchar        │
├──────────┼──────────┼─────────┼───┼──────────────────────┼──────────────────────┼──────────────────────┤
│   100654 │ 00100200 │ 001002  │ … │ 251                  │ 78                   │ 4900 Meridian Street │
│   100663 │ 00105200 │ 001052  │ … │ 1182                 │ 1176                 │ Administration Bld…  │
│   100690 │ 02503400 │ 025034  │ … │ 2                    │ 24                   │ 1200 Taylor Rd       │
│   100706 │ 00105500 │ 001055  │ … │ 510                

In [16]:
recent_cohorts = data_dir_path / 'data' / 'Most-Recent-Cohorts-Institution.csv'
df = pd.read_csv(recent_cohorts)
df

/var/folders/fk/07csw28s7qj2xcz4_f9th0hh0000gn/T/ipykernel_22886/932353004.py:2: DtypeWarning: Columns (9,1537,1540,1542,1606,1608,1614,1615,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1703,1704,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984,2376,2377,2403,2404,2495,2496,2497,2498,2499,2500,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2527,2528,2529,2530,2958,3215,3231) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(recent_cohorts)


,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,OMAWDP8_NOPELL_FIRSTTIME,OMENRUP_NOPELL_FIRSTTIME,OMENRYP_NOPELL_NOTFIRSTTIME,OMENRAP_NOPELL_NOTFIRSTTIME,OMAWDP8_NOPELL_NOTFIRSTTIME,OMENRUP_NOPELL_NOTFIRSTTIME,OMACHT8_NOPELL_ALL,OMACHT8_NOPELL_FIRSTTIME,OMACHT8_NOPELL_NOTFIRSTTIME,ADDR
0,100654,100200.0,1002.0,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,...,0.3187,0.2709,0.0128,0.2949,0.4744,0.2179,329.0,251.0,78.0,4900 Meridian Street
1,100663,105200.0,1052.0,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,...,0.6937,0.0660,0.0111,0.2636,0.5136,0.2117,2358.0,1182.0,1176.0,Administration Bldg Suite 1070
2,100690,2503400.0,25034.0,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,https://www.amridgeuniversity.edu/,https://www2.amridgeuniversity.edu:9091/,...,0.0000,0.5000,0.0000,0.3333,0.4583,0.2083,26.0,2.0,24.0,1200 Taylor Rd
3,100706,105500.0,1055.0,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu/,finaid.uah.edu/,...,0.6471,0.0941,0.0082,0.2647,0.5948,0.1324,1122.0,510.0,612.0,301 Sparkman Dr
4,100724,100500.0,1005.0,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,...,0.4381,0.2167,0.0000,0.1444,0.3667,0.4889,510.0,420.0,90.0,915 S Jackson Street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6538,49576720,332918.0,3329.0,Pennsylvania State University-Penn State York,York,PA,17403-3326,Middle States Commission on Higher Education,york.psu.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6539,49576721,332919.0,3329.0,Pennsylvania State University-Penn State Great...,Malvern,PA,19355-1488,Middle States Commission on Higher Education,greatvalley.psu.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6540,49576722,332920.0,3329.0,Pennsylvania State University-Penn State Harri...,Middletown,PA,17057-4846,Middle States Commission on Higher Education,harrisburg.psu.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6541,49576723,332921.0,3329.0,Pennsylvania State University-Penn State Brand...,Media,PA,19063-5522,Middle States Commission on Higher Education,brandywine.psu.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df['INSTNM']

0                                Alabama A & M University
1                     University of Alabama at Birmingham
2                                      Amridge University
3                     University of Alabama in Huntsville
4                                Alabama State University
                              ...                        
6538        Pennsylvania State University-Penn State York
6539    Pennsylvania State University-Penn State Great...
6540    Pennsylvania State University-Penn State Harri...
6541    Pennsylvania State University-Penn State Brand...
6542    Burlington County Institute of Technology - Ad...
Name: INSTNM, Length: 6543, dtype: object